In [ ]:
import nbimporter
from math import floor

from s02_02_design_primary_energy_consumption import calc_E_C, calc_E_H
import s02_04_specific_builder_standard as section2_4
import s02_05_climate_adaptive_housing as section2_5
from s04_01_general import calc_cooling_load, calc_heating_load, get_virtual_heating_devices, calc_heating_mode

# 第二章　単位住戸の一次エネルギー消費量 / Chapter 2 Designed Primary Energy Consumption

## 第三節　基準一次エネルギー消費量 / Section 3 Standard primary energy consumption

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．基準一次エネルギー消費量 / Standard primary energy consumption

建築物エネルギー消費性能基準（H28 年 4 月以降）： 

Building energy consumption performance standard (after April, H28):

$$
\large 
E_{ST}^* = E_{SH} + E_{SC} + E_{SV} + E_{SL} + E_{SW} + E_{SM}
$$

建築物エネルギー消費性能基準（H28 年 4 月現存）： 

Building energy consumption performance standard (as of April, H28):

$$
\large 
E_{ST}^* = (E_{SH} + E_{SC} + E_{SV} + E_{SL} + E_{SW}) \times 1.1 + E_{SM}
$$

建築物のエネルギー消費性能の向上の一層の促進のために誘導すべき基準（H28 年 4 月以降）： 

Criteria to be derived to further promote the improvement of energy consumption performance of buildings (after April, H28):

$$
\large 
E_{ST}^* = (E_{SH} + E_{SC} + E_{SV} + E_{SL} + E_{SW}) \times 0.9 + E_{SM}
$$

建築物のエネルギー消費性能の向上の一層の促進のために誘導すべき基準（H28 年 4 月現存）： 

Criteria to be derived to further promote the improvement of the energy consumption performance of buildings (H28, as of April):

$$
\large 
E_{ST}^* = E_{SH} + E_{SC} + E_{SV} + E_{SL} + E_{SW} + E_{SM}
$$

$E_{ST}^*$：1 年当たりの基準一次エネルギー消費量(MJ/年) / Standard primary energy consumption per year   
$E_{SH}$：1 年当たりの暖房設備の基準一次エネルギー消費量(MJ/年) / Standard primary energy consumption of the heating system per year  
$E_{SC}$：1 年当たりの冷房設備の基準一次エネルギー消費量(MJ/年) / Primary primary energy consumption of cooling system per year  
$E_{SV}$：1 年当たりの機械換気設備の基準一次エネルギー消費量(MJ/年) / Standard primary energy consumption of mechanical ventilation equipment per year   
$E_{SL}$：1 年当たりの照明設備の基準一次エネルギー消費量(MJ/年) / Standard primary energy consumption of lighting fixtures per year  
$E_{SW}$：1 年当たりの給湯設備（コージェネレーション設備を含む）の基準一次エネルギー消費量(MJ/年) / Standard primary energy consumption of hot water supply equipment (including cogeneration equipment) per year  
$E_{SM}$：1 年当たりのその他の基準一次エネルギー消費量(MJ/年) / Other baseline primary energy consumption per year  

In [ ]:
def get_E_ST_star(E_SH, E_SC, E_SV, E_SL, E_SW, E_SM, reference):
    """基準一次エネルギー消費量

    :param E_SH: 1 時間当たりの暖房設備の基準一次エネルギー消費量
    :type E_SH: float
    :param E_SC: 1 時間当たりの冷房設備の基準一次エネルギー消費量
    :type E_SC: float
    :param E_SV: 1 時間当たりの換気設備の基準一次エネルギー消費量
    :type E_SV: float
    :param E_SL: 1 時間当たりの照明設備の基準一次エネルギー消費量
    :type E_SL: float
    :param E_SW: 1 時間当たりの給湯設備の基準一次エネルギー消費量
    :type E_SW: float
    :param E_SM: 1 時間当たりのその他の基準一次エネルギー消費量
    :type E_SM: float
    :param reference: 基準値計算仕様
    :type reference: dict
    :return: 基準一次エネルギー消費量
    :rtype: float
    """
    result_type = reference['result_type']
    
    if result_type == '建築物エネルギー消費性能基準（H28 年 4 月以降）':
        E_star_ST = E_SH + E_SC + E_SV + E_SL + E_SW + E_SM  # (1a)
    elif result_type == '建築物エネルギー消費性能基準（H28 年 4 月現存）':
        E_star_ST = (E_SH + E_SC + E_SV + E_SL + E_SW) * 1.1 + E_SM  # (1b)
    elif result_type == '建築物のエネルギー消費性能の向上の一層の促進のために誘導すべき基準（H28 年 4 月以降）':
        E_star_ST = (E_SH + E_SC + E_SV + E_SL + E_SW) * 0.9 + E_SM  # (1c)
    elif result_type == '建築物のエネルギー消費性能の向上の一層の促進のために誘導すべき基準（H28 年 4 月現存）':
        E_star_ST = E_SH + E_SC + E_SV + E_SL + E_SW + E_SM  # (1d)
    elif result_type == '特定建築主基準':
        E_star_ST = section2_4.get_E_ST_star(E_SH, E_SC, E_SV, E_SL, E_SW, E_SM, reference)
    elif result_type == '気候風土適応住宅 新築' or result_type == '気候風土適応住宅 平成28年4月1日時点で現存':
        E_star_ST = section2_5.get_E_ST_star(E_SH, E_SC, E_SV, E_SL, E_SW, E_SM, result_type)
    else:
        raise ValueError('result_type')

    return E_star_ST


In [ ]:
def calc_heating_refernce_spec(region, mode_H, H_MR, H_OR):
    """暖房設備の基準一次エネルギー消費量
    
    :param region: 省エネルギー地域区分
    :type region: int
    :param mode_H: 暖房方式
    :type mode_H: str
    :param H_MR: 暖房機器の仕様
    :type H_MR: dict
    :param H_OR: 暖房機器の仕様
    :type H_OR: dict
    :raises ValueError: mode_MRが「ろ」かつregion が 1-8 でない場合に発生
    :raises ValueError: mode_MRが「は」かつregion が 1-8 でない場合に発生
    :raises ValueError: mode_MRが「ろ」または「は」以外の場合に発生
    :raises ValueError: mode_ORが「ろ」かつregion が 1-8 でない場合に発生
    :raises ValueError: mode_ORが「は」かつregion が 1-8 でない場合に発生
    :raises ValueError: mode_ORが「ろ」または「は」以外の場合に発生
    :raises ValueError: mode_Hが「住戸全体を連続的に暖房する方式」または「」
    :return: mode_H, H_A, H_MR, H_OR, H_HS, mode_MR, mode_OR (暖房方式及び運転方法の区分)
    :rtype: tuple
    """
    if region == 8:
        return None, None, None, None, None, None, None

    if mode_H == '住戸全体を連続的に暖房する方式':
        H_A = {
            'type': 'ダクト式セントラル空調機',
            'duct_insulation': '全てもしくは一部が断熱区画外である',
            'VAV': False,
            'general_ventilation': True,
            'EquipmentSpec': '入力しない'
        }
        H_MR = None
        H_OR = None
        H_HS = None
    elif mode_H == '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合' or \
            mode_H == '設置しない' or mode_H is None:

        # 実質的な暖房機器の仕様を取得
        spec_MR, spec_OR = get_virtual_heating_devices(region, H_MR, H_OR)
        if spec_MR is None and spec_OR is None:
            return None, None, None, None, None, None, None

        # 暖房方式及び運転方法の区分
        mode_MR, mode_OR = calc_heating_mode(region=region, H_MR=spec_MR, H_OR=spec_OR)

        H_A = None
        H_HS = None

        if mode_MR == 'ろ':
            if region in [1,2,3,4]:
                H_MR = {
                    'type': '温水暖房用パネルラジエーター',
                }
                H_HS = {
                    'type': '石油従来型温水暖房機',
                    'e_rtd_hs': 0.83,
                    'pipe_insulation': True,
                    'underfloor_pipe_insulation': False
                }
            elif region in [5,6,7,8]:
                H_MR = {
                    'type': '温水暖房用パネルラジエーター',
                }
                H_HS = {
                    'type': 'ガス潜熱回収型温水暖房機',
                    'e_rtd_hs': 0.825,
                    'pipe_insulation': True,
                    'underfloor_pipe_insulation': False
                }
            else:
                raise ValueError(region)
        elif mode_MR == 'は':
            if region in [1,2,3,4]:
                H_MR = {
                    'type': 'FF暖房機',
                    'e_rtd': 0.86
                }
            elif region in [5,6,7,8]:
                H_MR = {
                    'type': 'ルームエアコンディショナー',
                    'e_class': 'ろ',
                    'dualcompressor': False
                }
            else:
                raise ValueError(region)
        else:
            raise ValueError(mode_MR)

        if mode_OR == 'ろ':
            if region in [1,2,3,4]:
                H_OR = {
                    'type': '温水暖房用パネルラジエーター',
                }
                H_HS = {
                    'type': '石油従来型温水暖房機',
                    'e_rtd_hs': 0.83,
                    'pipe_insulation': True,
                    'underfloor_pipe_insulation': False
                }
            elif region in [4,5,6,7]:
                H_OR = {
                    'type': '温水暖房用パネルラジエーター',
                }
                H_HS = {
                    'type': 'ガス潜熱回収型温水暖房機',
                    'e_rtd_hs': 0.825,
                    'pipe_insulation': True,
                    'underfloor_pipe_insulation': False
                }
            else:
                raise ValueError(region)
        elif mode_OR == 'は':
            if region in [1,2,3,4]:
                H_OR = {
                    'type': 'FF暖房機',
                    'e_rtd': 0.86
                }
            elif region in [5,6,7,8]:
                H_OR = {
                    'type': 'ルームエアコンディショナー',
                    'e_class': 'ろ',
                    'dualcompressor': False
                }
            else:
                raise ValueError(region)
        elif mode_OR is None:
            pass
        else:
            raise ValueError(mode_OR)
    else:
        raise ValueError(mode_H)



    if mode_H is None:
        mode_H = '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合'

    # 暖房方式及び運転方法の区分
    mode_MR, mode_OR = calc_heating_mode(region=region, H_MR=H_MR, H_OR=H_OR)

    return mode_H, H_A, H_MR, H_OR, H_HS, mode_MR, mode_OR


In [ ]:
def calc_env_reference_spec(type, tatekata, region, reference, A_A, ENV):
    """熱損失係数, 熱取得率（暖房）, 熱取得率（冷房）を計算する
    
    :param type: [description]
    :type type: str
    :param tatekata: 建て方
    :type tatekata: str
    :param region: 省エネルギー地域区分
    :type region: int
    :param reference: 基準計算仕様
    :type reference: dict
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param ENV: [description]
    :type ENV: dict
    :return: 熱損失係数, 熱取得率（暖房）, 熱取得率（冷房）
    :rtype: tuple
    """
    from s03_01_hc_load_general import get_Q

    if ENV is None:
        return None, None, None

    if type != '行政庁認定住宅':
        from s02_03_a_settings_of_hc_loads_and_shell_performance import get_U_A, get_etr_A_H, get_etr_A_C

        U_A = get_U_A(tatekata, region, reference)
        etr_A_H = get_etr_A_H(tatekata, region, reference)
        etr_A_C = get_etr_A_C(tatekata, region, reference)

        from s03_02_envelope_performance import get_Q_dash, get_eta_H, get_eta_C

        if ENV['method'] == '当該住宅の外皮面積の合計を用いて評価する':
            A_env = ENV['A_env']
            r_env = A_env / A_A  # 床面積の合計に対する外皮の部位の面積の合計の比
        elif ENV['method'] == '簡易的に求めた外皮面積の合計を用いて評価する':
            from s03_02_09_envelope_performance_simple import get_A_dash_env, get_A_dash_A, get_r_env
            A_dash_env = get_A_dash_env(ENV['house_insulation_type'])
            A_dash_A = get_A_dash_A()
            r_env = get_r_env(A_dash_env, A_dash_A)
        elif ENV['method'] == '当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法':
            from s03_02_09_envelope_performance_simple import get_A_dash_env, get_A_dash_A, get_r_env
            from s03_02_envelope_performance import calc_insulation_performance
            _, _, _, _, _, _, _, house_insulation_type = calc_insulation_performance(**ENV)
            A_dash_env = get_A_dash_env(house_insulation_type)
            A_dash_A = get_A_dash_A()
            r_env = get_r_env(A_dash_env, A_dash_A)

        Q_dash = get_Q_dash(U_A, r_env)

        eta_H = get_eta_H(etr_A_H, r_env)
        eta_C = get_eta_C(etr_A_C, r_env)
    else:
        from s03_02_envelope_performance import calc_insulation_performance
        U_A, r_env, eta_A_H, eta_A_C, Q_dash, eta_H, eta_C,_ = calc_insulation_performance(**ENV)

    Q = get_Q(Q_dash)

    return Q, eta_H, eta_C


### 6. 暖房設備の基準一次エネルギー消費量 / Standard primary energy consumption of heating equipment

$$
\large 
E_{SH} = \sum_{d=1}^{365} \sum_{t=0}^{23} E_{SH,d,t}
$$

$E_{SH,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の基準一次エネルギー消費量(MJ/h) / Standard primary energy consumption of heating equipment per hour at time $ t $ on date $ d $  

日付$d$の時刻$t$における1 時間当たりの暖房設備の基準一次エネルギー消費量$E_{SH,d,t}$は、地域の区分が 8地域の場合は0とし、それ以外の場合は、式により表される。 

The standard primary energy consumption $ E_ {SH, d, t} $ of the heating equipment per hour at the time $ t $ of the date $ d $ is 0 if the area division is 8 areas, otherwise Is represented by a formula.

$$
\large 
E_{SH,d,t} = E_{E,SH,d,t} \times f_{prim} \times 10^{-3} + E_{G,SH,d,t} + E_{K,SH,d,t} + E_{M,SH,d,t} + E_{UT,SH,d,t}
$$

$E_{E,SH,d,t}$：日付$d$の時刻$t$における 1 時間当たりの暖房設備の消費電力量(kWh/h) / Power consumption of heating equipment per hour at time $ t $ on date $ d $  
$f_{prim}$：電気の量 1kWh を熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / A factor to convert 1kWh of electricity into heat quantity (Chapter 2, Section 1 Appendix B)  
$E_{G,SH,d,t}$：日付$d$の時刻$t$における 1 時間当たりの暖房設備のガス消費量(MJ/h) / Gas consumption of heating system per hour at time $ t $ on date $ d $  
$E_{K,SH,d,t}$：日付$d$の時刻$t$における 1 時間当たりの暖房設備の灯油消費量(MJ/h) / Kerosene consumption of heating system per hour at time $ t $ on date $ d $  
$E_{M,SH,d,t}$：日付$d$の時刻$t$における 1 時間当たりの暖房設備のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by heating unit other fuel per hour at time $ t $ on date $ d $  
$E_{UT,SH,d,t}$：日付$d$の時刻$t$における 1 時間当たりの暖房設備の未処理暖房負荷の基準一次エネルギー消費量相当値(MJ/h) / Reference primary energy consumption equivalent value of the heating equipment's raw heating load per hour at time $ t $ on date $ d $  

In [ ]:
def calc_E_SH(type, tatekata, region, reference, sol_region, A_A, A_MR, A_OR, ENV, mode_H, mode_C, NV_MR, NV_OR, H_MR, H_OR):
    """暖房設備の設計一次エネルギー消費量
    
    :param type: [description]
    :type type: str
    :param tatekata: 建て方
    :type tatekata: str
    :param region: 省エネルギー地域区分
    :type region: int
    :param reference: 基準計算仕様
    :type reference: dict
    :param sol_region: 年間の日射地域区分(1-5)
    :type sol_region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param ENV: [description]
    :type ENV: dict
    :param mode_H: 暖房方式
    :type mode_H: str
    :param mode_C: 冷房方式
    :type mode_C: str
    :param NV_MR: 主たる居室における通風の利用における相当換気回数
    :type NV_MR: float
    :param NV_OR: その他の居室における通風の利用における相当換気回数
    :type NV_OR: float
    :param H_MR: 暖房機器の仕様
    :type H_MR: dict
    :param H_OR: 暖房機器の仕様
    :type H_OR: dict
    :return: 暖房設備の設計一次エネルギー消費量
    :rtype: float
    """
    # 暖房基準値計算用仕様の取得
    mode_H, H_A, H_MR, H_OR, H_HS, mode_MR, mode_OR = calc_heating_refernce_spec(region, mode_H, H_MR, H_OR)

    # 外皮の基準値計算用仕様の取得
    Q, eta_H, eta_C = calc_env_reference_spec(type, tatekata, region, reference, A_A, ENV)

    # 暖房負荷の取得
    L_T_H_d_t_i, L_dash_H_R_d_t_i = calc_heating_load(
        region, sol_region,
        A_A, A_MR, A_OR,
        Q, eta_H, eta_C, NV_MR, NV_OR, None, None, None,
        None, mode_H, mode_C,
        H_MR, H_OR, mode_MR, mode_OR, None)

    L_CS_d_t_i, L_CL_d_t_i = calc_cooling_load(
        region, A_A, A_MR, A_OR,
        Q, eta_H, eta_C, NV_MR, NV_OR, None, None,
        mode_C, mode_H, mode_MR, mode_OR, None, None)

    if (ENV != None) and ('A_env' in ENV):
        A_env = ENV['A_env']
    else:
        A_env = None

    E_SH = calc_E_H(region, sol_region, A_A, A_MR, A_OR, A_env, eta_H, eta_C, Q, mode_H,
                  H_A, H_MR, H_OR, H_HS, mode_MR, mode_OR, None, None,
                  None, L_T_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i)

    return E_SH


### 7. 冷房設備の基準一次エネルギー消費量 / Standard primary energy consumption of cooling system

$$
\large 
E_{SC} = \sum_{d=1}^{365} \sum_{t=0}^{23} E_{SC,d,t}
$$

$E_{SC,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の基準一次エネルギー消費量(MJ/h) / Basic primary energy consumption of cooling system per hour at time $ t $ of date $ d $  

$$
\large 
E_{SC,d,t} = E_{E,SC,d,t} \times f_{prim} \times 10^{-3} + E_{G,SC,d,t} + E_{K,SC,d,t} + E_{M,SC,d,t}
$$

$E_{E,SC,d,t}$：日付$d$の時刻$t$における 1 時間当たりの冷房設備の消費電力量(kWh/h) / Power consumption of cooling system per hour at time $ t $ on date $ d $   
$f_{prim}$：電気の量1kWhを熱量に換算する係数（第二章第一節付録 B）(kJ/kWh) / A factor to convert 1 kWh of electricity into heat quantity (Chapter 2, Section 1 Appendix B)  
$E_{G,SC,d,t}$：日付$d$の時刻$t$における 1 時間当たりの冷房設備のガス消費量(MJ/h) / Gas consumption of cooling system per hour at time $ t $ on date $ d $   
$E_{K,SC,d,t}$：日付$d$の時刻$t$における 1 時間当たりの冷房設備の灯油消費量(MJ/h) / Kerosene consumption of cooling system per hour at time $ t $ on date $ d $  
$E_{M,SC,d,t}$：日付$d$の時刻$t$における 1 時間当たりの冷房設備のその他の燃料による一次エネルギー消費量(MJ/h) / Primary energy consumption by cooling system other fuel per hour at time $ t $ on date $ d $  

In [ ]:
def calc_E_SC(type, tatekata, region, reference, A_A, A_MR, A_OR, ENV, mode_C, mode_H, H_MR, H_OR, TS, HEX,
              sol_region, NV_MR, NV_OR):
    """1 年当たりの冷房設備の設計一次エネルギー消費量
    
    :param type: [description]
    :type type: str
    :param tatekata: 建て方
    :type tatekata: str
    :param region: 省エネルギー地域区分
    :type region: int
    :param reference: 基準計算仕様
    :type reference: dict
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param ENV: [description]
    :type ENV: dict
    :param mode_C: 冷房方式
    :type mode_C: str
    :param mode_H: 暖房方式
    :type mode_H: str
    :param H_MR: 暖房機器の仕様
    :type H_MR: dict
    :param H_OR: 暖房機器の仕様
    :type H_OR: dict
    :param TS: 蓄熱の利用
    :type TS: bool
    :param HEX: 熱交換器型設備仕様辞書
    :type HEX: dict
    :raises ValueError: mode_C が　「住戸全体を連続的に冷房する方式」または「居室のみを冷房する方式」でない場合に発生する
    :return: 1 年当たりの冷房設備の設計一次エネルギー消費量
    :rtype: float
    """
    if ENV is None:
        return 0.0

    if mode_C == '住戸全体を連続的に冷房する方式':
        C_A = {
            'type': 'ダクト式セントラル空調機',
            'duct_insulation': '全てもしくは一部が断熱区画外である',
            'VAV': False,
            'general_ventilation': True,
            'EquipmentSpec': '入力しない'
        }
        C_MR = None
        C_OR = None
    elif mode_C == '居室のみを冷房する方式' or mode_C == '設置しない' or mode_C is None:
        C_A = None
        C_MR = {
            'type': 'ルームエアコンディショナー',
            'e_class': 'ろ',
            'dualcompressor': False
        }
        C_OR = {
            'type': 'ルームエアコンディショナー',
            'e_class': 'ろ',
            'dualcompressor': False
        }
    else:
        raise ValueError(mode_C)

    if mode_C is None:
        mode_C = '居室のみを冷房する方式'

    # 暖房基準値計算用仕様の取得
    mode_H, H_A, H_MR, H_OR, H_HS, mode_MR, mode_OR = calc_heating_refernce_spec(region, mode_H, H_MR, H_OR)

    # 外皮の基準値計算用仕様の取得
    Q, eta_H, eta_C = calc_env_reference_spec(type, tatekata, region, reference, A_A, ENV)

    # 暖房負荷の取得
    L_T_H_d_t_i, L_dash_H_R_d_t_i = calc_heating_load(
        region, sol_region,
        A_A, A_MR, A_OR,
        Q, eta_H, eta_C, NV_MR, NV_OR, None, None, None,
        None, mode_H, mode_C,
        H_MR, H_OR, mode_MR, mode_OR, None)

    # 冷房負荷の計算
    L_CS_d_t, L_CL_d_t = calc_cooling_load(
        region=region,
        A_A=A_A,
        A_MR=A_MR,
        A_OR=A_OR,
        Q=Q,
        mu_C=eta_C,
        mu_H=eta_H,
        # 通風なし
        NV_MR=0,
        NV_OR=0,
        # 床下換気なし
        r_A_ufvnt=None,
        underfloor_insulation=None,
        mode_C=mode_C,
        mode_H=mode_H,
        mode_MR=mode_MR,
        mode_OR=mode_OR,
        TS=TS,
        HEX=HEX
    )

    if 'A_env' in ENV:
        A_env = ENV['A_env']
    else:
        A_env = None

    # 1 年当たりの冷房設備の設計一次エネルギー消費量
    E_SC = calc_E_C(region, A_A, A_MR, A_OR, A_env, eta_H, eta_C, Q, C_A, C_MR, C_OR, L_T_H_d_t_i, L_CS_d_t, L_CL_d_t, mode_C)

    return E_SC


### 8. 機械換気設備の基準一次エネルギー消費量 / Standard primary energy consumption of mechanical ventilation equipment

$$
\large 
E_{SV} = \alpha_{SV} A_{A} + \beta_{SV}
$$

$A_{A}$：床面積の合計(m<sup>2</sup>) / Total floor area  
$\alpha_{SV}$：係数(MJ/(m<sup>2</sup>・年)) / coefficient  
$\beta_{SV}$：係数(MJ/年) / coefficient  

In [ ]:
def calc_E_SV(A_A):
    """ 1年あたりの機械換気設備の基準一次エネルギー消費量 (6)

    :param A_A: 床面積の合計(付録Aによる定まる値) (m2)
    :type A_A: float
    :return: 1年あたりの機械換気設備の基準一次エネルギー消費量 (MJ/年)
    :rtype: float
    """
    # 係数 a_SV, b_SV
    a_SV, b_SV = get_table3_coeff(A_A)

    # 1年あたりの機械換気設備の基準一次エネルギー消費量 (9)
    E_SV = get_E_SV(A_A, a_SV, b_SV)

    return E_SV


In [ ]:
def get_E_SV(A_A, a_SV, b_SV):
    """ 1年あたりの機械換気設備の基準一次エネルギー消費量 (6)
    :param A_A: 床面積の合計 (m2)
    :param a_SV: 係数 (MJ/(m2・年))
    :param b_SV: 係数 (MJ/年)
    :return: 1年あたりの機械換気設備の基準一次エネルギー消費量 (MJ/年)
    """
    E_SV = a_SV * A_A + b_SV
    return E_SV


係数$\alpha_{SV}$および$\beta_{SV}$は、床面積の合計$A_{A}$に応じて表により定まる。 

The coefficients $ \alpha_ {SV} $ and $ \beta_ {SV} $ are tabulated according to the total floor area $ A_ {A} $.

<div style="text-align: center;font-weight: bold;">表　機械換気設備の一次エネルギー消費量の算出に用いる係数$\alpha_{SV}$・$\beta_{SV}$<br>Table: Coefficients used to calculate primary energy consumption of mechanical ventilation equipment $ \alpha_{SV} $ ・ $ \beta_{SV} $</div>

<table>
<thead>
	<tr>
        <th rowspan="2">係数<br>coefficient</th>
		<th colspan="3" style="text-align: center;">床面積の合計$A_{A}$の区分<br>Division of floor space total $ A_ {A} $</th>
	</tr>
	<tr>
		<th>（い）床面積の合計が30m<sup>2</sup>未満<br>(I) total floor area is less than 30m<sup>2</sup></th>
		<th>（ろ）床面積の合計が30m<sup>2</sup>以上 120m<sup>2</sup>未満<br>(RO) The total floor area is<br>more than 30m<sup>2</sup> and less than 120m<sup>2</sup></th>
		<th>（は）床面積の合計が120m<sup>2</sup>以上<br>(HA)Total floor area is over 120m<sup>2</sup></th>
	</tr>
</thead>
<tbody>
	<tr>
		<th>$\alpha_{SV}$</th>
		<td>33</td>
		<td>38</td>
		<td>33</td>
	</tr>
    <tr>
		<th>$\beta_{SV}$</th>
		<td>129</td>
		<td>-21</td>
		<td>579</td>
	</tr>
</tbody>
</table>

In [ ]:
def get_table3_coeff(A_A):
    """その他の一次エネルギー消費量の算出に用いる表3の係数を取得

    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :return: 係数 a_SV, b_SV
    :rtype: tuple
    """

    # その他の一次エネルギー消費量の算出に用いる係数
    table_3 = [
        (33, 38, 33),
        (129, -21, 579)
    ]
    if A_A < 30:
        index = 0
    elif A_A < 120:
        index = 1
    else:
        index = 2

    a_SV = table_3[0][index]
    b_SV = table_3[1][index]

    return a_SV, b_SV


### 9. 照明設備の基準一次エネルギー消費量 / Standard Primary Energy Consumption of Lighting Fixtures

$$
\large 
E_{SL} = 31 \times A_{A} + 169 \times A_{MR} + 39 \times A_{OR}
$$

$A_{A}$：床面積の合計(m<sup>2</sup>) / total floor area  
$A_{MR}$：主たる居室の床面積(m<sup>2</sup>) / Floor area of main room  
$A_{OR}$：その他の居室の床面積(m<sup>2</sup>) / Floor area of other rooms  

In [ ]:
def calc_E_SL(A_A, A_MR, A_OR):
    """1年あたりの照明の基準一次エネルギー消費量 (7)

    :param A_A: 床面積の合計(付録Aによる定まる値) (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積の合計(付録Aによる定まる値) (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積の合計(付録Aによる定まる値) (m2)
    :type A_OR: float
    :return: 1年あたりの照明の基準一次エネルギー消費量 (MJ/年)
    :rtype: float
    """
    E_SL = get_E_SL(A_A, A_MR, A_OR)
    return E_SL


In [ ]:
def get_E_SL(A_A, A_MR, A_OR):
    """1年あたりの照明の基準一次エネルギー消費量 (7)

    :param A_A: 床面積の合計(付録Aによる定まる値) (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積の合計(付録Aによる定まる値) (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積の合計(付録Aによる定まる値) (m2)
    :type A_OR: float
    :return: 1年あたりの照明の基準一次エネルギー消費量 (MJ/年)
    :rtype: float
    """
    E_SL = 31 * A_A + 169 * A_MR + 39 * A_OR
    return E_SL


### 10. 給湯設備及びコージェネレーション設備の基準一次エネルギー消費量 / Standard primary energy consumption of hot water supply equipment and cogeneration equipment

浴室等、台所及び洗面所が無い場合、1 年当たりの給湯設備（コージェネレーション設備を含む）の基準一次エネルギー消費量$E_{SW}$は、ゼロとする。

If there is no kitchen and bathroom, etc., the standard primary energy consumption $ E_ {SW} $ of hot water supply equipment (including cogeneration equipment) per year is zero.

浴室等が有る場合、もしくは浴室等が無く、台所又は洗面所が有る場合、

If there is a bathroom, etc., or if there is no bathroom, and there is a kitchen or a lavatory,

$$
\large 
E_{SW} = \alpha_{SW} A_{A} + \beta_{SW}
$$

$A_{A}$：床面積の合計(m<sup>2</sup>) / total floor area  
$\alpha_{SW}$：係数(MJ/(m<sup>2</sup>・年)) / coefficient  
$\beta_{SW}$：係数(MJ/年) / coefficient  

In [ ]:
def calc_E_SW(region, A_A, HW):
    """1年当たりの給湯設備の設計一次エネルギー消費量 (8)
    
    :param region: 省エネルギー地域区分
    :type region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param HW: 給湯機の仕様
    :type HW: dict
    :return: 1年当たりの給湯設備の設計一次エネルギー消費量 (8)
    :rtype: float
    """
    if HW is None or HW['hw_type'] is None:
        return 0.0
    else:
        a_SW, b_SW = get_table4_coeff(region, A_A, HW['has_bath'])
        E_SW = get_E_SW(A_A, a_SW, b_SW)
        return E_SW


In [ ]:
def get_E_SW(A_A, a_SW, b_SW):
    """1年当たりの給湯設備の設計一次エネルギー消費量 (8)
    
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param a_SW: 表4 給湯設備の一次エネルギー消費量の算出に用いる係数
    :type a_SW: float
    :param b_SW: 表4 給湯設備の一次エネルギー消費量の算出に用いる係数
    :type b_SW: float
    :return: 1年当たりの給湯設備の設計一次エネルギー消費量 (8)
    :rtype: float
    """
    if a_SW is None:
        E_SW = b_SW
    else:
        E_SW = a_SW * A_A + b_SW
    return E_SW


<div style="text-align: center;font-weight: bold;">表　給湯設備の一次エネルギー消費量の算出に用いる係数$\alpha_{SW}$・$\beta_{SW}$<br>Table: Factors $ \alpha_ {SW} $ ・ $ \beta_ {SW} $ used to calculate the primary energy consumption of water heaters</div>

<table>
<thead>
	<tr>
        <th rowspan="2">地域の区分<br>Segmentation of area</th>
        <th rowspan="2">給湯対象室<br>Hot water supply target room</th>
        <th rowspan="2">係数<br>coefficient</th>
		<th colspan="5" style="text-align: center;">床面積の合計$A_{A}$の区分<br>Division of floor space total $ A_ {A} $</th>
	</tr>
	<tr>
		<th>（い）<br>床面積の<br>合計が<br>30m<sup>2</sup>未満<br>(I) total floor area is less than 30m<sup>2</sup></th>
		<th>（ろ）<br>床面積の<br>合計が<br>30m<sup>2</sup>以上<br>60m<sup>2</sup>未満<br>(RO)total floor area is more than 30m<sup>2</sup> and less than 60m<sup>2</sup></th>
		<th>（は）<br>床面積の<br>合計が<br>60m<sup>2</sup>以上<br>90m<sup>2</sup>未満<br>(HA)Total floor area is 60m<sup>2</sup> or more and less than 90m<sup>2</sup></th>
		<th>（に）<br>床面積の<br>合計が<br>90m<sup>2</sup>以上<br>120m<sup>2</sup>未満<br>(NI)Total floor area is 90m<sup>2</sup> or more and less than 120m<sup>2</sup></th>
		<th>（ほ）<br>床面積の<br>合計が<br>120m<sup>2</sup>以上<br>(HO)Total floor area is 120m<sup>2</sup></th>
	</tr>
</thead>
    
<tbody>
	<tr>
        <th rowspan="4">1</th>
        <th rowspan="2">浴室等が有る場合<br>When there is bathroom</th>
		<th>$\alpha_{SW}$</th>
		<td>－</td><td>234</td><td>307</td><td>109</td><td>－</td>
	</tr>
    <tr>
		<th>$\beta_{SW}$</th>
		<td>11946</td><td>4926</td><td>546</td><td>18366</td><td>31446</td>
	</tr>
    <tr>
        <th rowspan="2">浴室等が無く、台所<br>又は洗面が有る場合<br>When there is no bathroom, and there is a kitchen or bathroom</th>
		<th>$\alpha_{SW}$</th>
		<td>－</td><td>32</td><td>78</td><td>15</td><td>－</td>
	</tr>
    <tr>
		<th>$\beta_{SW}$</th>
		<td>4835</td><td>3875</td><td>1115</td><td>6785</td><td>8585</td>
	</tr>
    <tr>
        <th rowspan="4">2</th>
        <th rowspan="2">浴室等が有る場合<br>When there is bathroom</th>
		<th>$\alpha_{SW}$</th>
		<td>－</td><td>228</td><td>300</td><td>107</td><td>－</td>
	</tr>
    <tr>
		<th>$\beta_{SW}$</th>
		<td>11696</td><td>4856</td><td>536</td><td>17906</td><td>30746</td>
	</tr>
    <tr>
        <th rowspan="2">浴室等が無く、台所<br>又は洗面が有る場合<br>When there is no bathroom, and there is a kitchen or bathroom</th>
		<th>$\alpha_{SW}$</th>
		<td>－</td><td>32</td><td>77</td><td>15</td><td>－</td>
	</tr>
    <tr>
		<th>$\beta_{SW}$</th>
		<td>4742</td><td>3782</td><td>1082</td><td>6662</td><td>8462</td>
	</tr>
    <tr>
        <th rowspan="4">3</th>
        <th rowspan="2">浴室等が有る場合<br>When there is bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>212</td><td>280</td><td>100</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>10892</td><td>4532</td><td>452</td><td>16652</td><td>28652</td>
    </tr>
    <tr>
        <th rowspan="2">浴室等が無く、台所<br>又は洗面が有る場合<br>When there is no bathroom, and there is a kitchen or bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>30</td><td>72</td><td>14</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>4442</td><td>3542</td><td>1022</td><td>6242</td><td>7922</td>
    </tr>
    <tr>
        <th rowspan="4">4</th>
        <th rowspan="2">浴室等が有る場合<br>When there is bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>205</td><td>272</td><td>97</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>10575</td><td>4425</td><td>405</td><td>16155</td><td>27795</td>
    </tr>
    <tr>
        <th rowspan="2">浴室等が無く、台所<br>又は洗面が有る場合<br>When there is no bathroom, and there is a kitchen or bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>29</td><td>70</td><td>13</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>4321</td><td>3451</td><td>991</td><td>6121</td><td>7681</td>
    </tr>
    <tr>
        <th rowspan="4">5</th>
        <th rowspan="2">浴室等が有る場合<br>When there is bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>200</td><td>276</td><td>103</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>10440</td><td>4440</td><td>-120</td><td>15450</td><td>27810</td>
    </tr>
    <tr>
        <th rowspan="2">浴室等が無く、台所<br>又は洗面が有る場合<br>When there is no bathroom, and there is a kitchen or bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>29</td><td>71</td><td>14</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>4165</td><td>3295</td><td>775</td><td>5905</td><td>7585</td>
    </tr>
    <tr>
        <th rowspan="4">6</th>
        <th rowspan="2">浴室等が有る場合<br>When there is bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>181</td><td>249</td><td>93</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>9401</td><td>3971</td><td>-109</td><td>13931</td><td>25091</td>
    </tr>
    <tr>
        <th rowspan="2">浴室等が無く、台所<br>又は洗面が有る場合<br>When there is no bathroom, and there is a kitchen or bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>26</td><td>64</td><td>12</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>3755</td><td>2975</td><td>695</td><td>5375</td><td>6815</td>
    </tr>
    <tr>
        <th rowspan="4">7</th>
        <th rowspan="2">浴室等が有る場合<br>When there is bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>165</td><td>227</td><td>85</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>8499</td><td>3549</td><td>-171</td><td>12609</td><td>22809</td>
    </tr>
    <tr>
        <th rowspan="2">浴室等が無く、台所<br>又は洗面が有る場合<br>When there is no bathroom, and there is a kitchen or bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>23</td><td>57</td><td>11</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>3402</td><td>2712</td><td>672</td><td>4812</td><td>6132</td>
    </tr>
    <tr>
        <th rowspan="4">8</th>
        <th rowspan="2">浴室等が有る場合<br>When there is bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>130</td><td>178</td><td>67</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>6672</td><td>2772</td><td>-108</td><td>9882</td><td>17922</td>
    </tr>
    <tr>
        <th rowspan="2">浴室等が無く、台所<br>又は洗面が有る場合<br>When there is no bathroom, and there is a kitchen or bathroom</th>
        <th>$\alpha_{SW}$</th>
        <td>－</td><td>18</td><td>45</td><td>9</td><td>－</td>
    </tr>
    <tr>
        <th>$\beta_{SW}$</th>
        <td>2679</td><td>2139</td><td>519</td><td>3759</td><td>4839</td>
    </tr>
</tbody>
</table>

In [ ]:
def get_table4_coeff(region, A_A, has_bath):
    """表4 給湯設備の一次エネルギー消費量の算出に用いる係数
    
    :param region: 省エネルギー地域区分
    :type region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param has_bath: バスが存在するかどうか
    :type has_bath: bool
    :return: 給湯設備の一次エネルギー消費量の算出に用いる係数a_SW, b_SW
    :rtype: tuple
    """

    # 表4 給湯設備の一次エネルギー消費量の算出に用いる係数
    table_4 = [
        (None, 234, 307, 109, None),
        (11946, 4926, 546, 18366, 31446),
        (None, 32, 78, 15, None),
        (4835, 3875, 1115, 6785, 8585),
        (None, 228, 300, 107, None),
        (11696, 4856, 536, 17906, 30746),
        (None, 32, 77, 15, None),
        (4742, 3782, 1082, 6662, 8462),
        (None, 212, 280, 100, None),
        (10892, 4532, 452, 16652, 28652),
        (None, 30, 72, 14, None),
        (4442, 3542, 1022, 6242, 7922),
        (None, 205, 272, 97, None),
        (10575, 4425, 405, 16155, 27795),
        (None, 29, 70, 13, None),
        (4321, 3451, 991, 6121, 7681),
        (None, 200, 276, 103, None),
        (10440, 4440, -120, 15450, 27810),
        (None, 29, 71, 14, None),
        (4165, 3295, 775, 5905, 7585),
        (None, 181, 249, 93, None),
        (9401, 3971, -109, 13931, 25091),
        (None, 26, 64, 12, None),
        (3755, 2975, 695, 5375, 6815),
        (None, 165, 227, 85, None),
        (8499, 3549, -171, 12609, 22809),
        (None, 23, 57, 11, None),
        (3402, 2712, 672, 4812, 6132),
        (None, 130, 178, 67, None),
        (6672, 2772, -108, 9882, 17922),
        (None, 18, 45, 9, None),
        (2679, 2139, 519, 3759, 4839)
    ]

    row_index = (region - 1) * 4 + (0 if has_bath else 2)
    column_index = min(4, floor(A_A / 30))
    a_SW = table_4[row_index][column_index]
    b_SW = table_4[row_index + 1][column_index]
    return a_SW, b_SW


### 11. その他の基準一次エネルギー消費量 / Other standard primary energy consumption

$$
\large 
E_{SM} = \alpha_{SM} A_{A} + \beta_{SM}
$$

$A_{A}$：床面積の合計(m<sup>2</sup>) / total floor area  
$\alpha_{SM}$：係数(MJ/(m<sup>2</sup>・年)) / coefficient  
$\beta_{SM}$：係数(MJ/年) / coefficient  

In [ ]:
def calc_E_SM(A_A):
    """1年あたりのその他の基準一次エネルギー消費量 (9)

    :param A_A: 床面積の合計(付録Aによる定まる値) (m2)
    :type A_A: float
    :return: 1年あたりのその他の基準一次エネルギー消費量 (MJ/年)
    :rtype: float
    """
    # 係数 a_SM, b_SM
    a_SM, b_SM = get_table5_coeff(A_A)

    # 1年あたりのその他の基準一次エネルギー消費量 (9)
    E_SM = get_E_SM(A_A, a_SM, b_SM)

    return E_SM


In [ ]:
def get_E_SM(A_A, a_SM, b_SM):
    """1年あたりのその他の基準一次エネルギー消費量 (9)

    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param a_SM: 係数 (MJ/(m2・年))
    :type a_SM: float
    :param b_SM: 係数 (MJ/年)
    :type b_SM: float
    :return: 1年あたりのその他の基準一次エネルギー消費量 (MJ/年)
    :rtype: float
    """
    E_SM = a_SM * A_A + b_SM
    return E_SM


<div style="text-align: center;font-weight: bold;">表　その他の一次エネルギー消費量の算出に用いる係数$\alpha_{SM}$・$\beta_{SM}$<br>Table: Coefficients used to calculate other primary energy consumption $ \alpha_ {SM} $ ・ $ \beta_ {SM} $</div>

<table>
<thead>
	<tr>
        <th rowspan="2">係数<br>coefficient</th>
		<th colspan="5" style="text-align: center;">床面積の合計$A_{A}$の区分<br>Division of floor space total $ A_ {A} $</th>
	</tr>
	<tr>
		<th>（い）<br>床面積の<br>合計が<br>30m<sup>2</sup>未満<br>(I) total floor area is less than 30m<sup>2</sup></th>
		<th>（ろ）<br>床面積の<br>合計が<br>30m<sup>2</sup>以上<br>60m<sup>2</sup>未満<br>(RO)total floor area is more than 30m<sup>2</sup> and less than 60m<sup>2</sup></th>
		<th>（は）<br>床面積の<br>合計が<br>60m<sup>2</sup>以上<br>90m<sup>2</sup>未満<br>(HA)Total floor area is 60m<sup>2</sup> or more and less than 90m<sup>2</sup></th>
		<th>（に）<br>床面積の<br>合計が<br>90m<sup>2</sup>以上<br>120m<sup>2</sup>未満<br>(NI)Total floor area is 90m<sup>2</sup> or more and less than 120m<sup>2</sup></th>
		<th>（ほ）<br>床面積の<br>合計が<br>120m<sup>2</sup>以上<br>(HO)Total floor area is 120m<sup>2</sup></th>
	</tr>
</thead>
<tbody>
	<tr>
		<th>$\alpha_{SM}$</th>
		<td>0</td>
		<td>87.63</td>
		<td>166.71</td>
		<td>47.64</td>
		<td>0</td>
	</tr>
    <tr>
		<th>$\beta_{SM}$</th>
		<td>12181.13</td>
		<td>9552.23</td>
		<td>4807.43</td>
		<td>15523.73</td>
		<td>21240.53</td>
	</tr>
</tbody>
</table>

In [ ]:
def get_table5_coeff(A_A):
    """その他の一次エネルギー消費量の算出に用いる係数
    
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :return: 係数 a_SM, b_SM
    :rtype: tuple
    """
    # その他の一次エネルギー消費量の算出に用いる係数
    table_5 = [
        (0, 87.63, 166.71, 47.64, 0),
        (12181.13, 9552.23, 4807.43, 15523.73, 21240.53)
    ]

    index = min(4, floor(A_A / 30))
    a_SM = table_5[0][index]
    b_SM = table_5[1][index]
    return a_SM, b_SM
